In [1]:
import pandas as pd
import codecs as cd
import glob
import xlsxwriter
import os
from dateutil import parser
import re

## 1. Commond Function

Mục đích : Tạo ra các function dùng chung cho toàn bộ project

In [2]:
def to_number(str):
    '''
        Input : String
        Output : Float. If Null values will be return None
    '''
    temp = None
    try:
        temp = float(str)
    except ValueError:
        return None
    return temp

def to_date(column):
    '''
        Input : Pandas Serial
    '''
    tmp_date = []
    for x in column:
        try:
            tmp_date.append(parse(str(x)))
        except ValueError:
            tmp_date.append(None)
    return tmp_date

def is_date(in_data):
    # Kiểm tra str có phải dạng datetime không
    pattern1 = r'^\d\d\d\d/\d\d/\d\d$'
    pattern2 = r'^\d\d\/\d\d/\d\d$'
    pattern3 = r'^\d\d\d\d\/\d\d\/\d\d \d\d\:\d\d$'
    pattern4 = r'^\d\d\/\d\d\/\d\d \d\d\:\d\d$'
    pattern = [pattern1, pattern2, pattern3, pattern4]

    for pt in pattern:
        if re.match(pt, in_data) is not None:
            return True
    return False

def is_num(in_data):
    return in_data.replace('.','',1).isdigit()

## 2. Get folder and file info

Ý nghĩa các class : 
* file_info : Input (URL folder chứa data csv). Output (filename, extension, file size, file content dạng DataFrame)
* read_from_folder : Input (URL). Output (Trả về thông tin của toàn bộ các file trong folder + Nội dung toàn bộ file content được lưu trong Panel Pandas) 

In [52]:
class file_info(object):
    def get_fileName(url):
        return url[url.rfind('\\') + 1 :]

    def get_Extension(url):
        return url[url.rfind('.') + 1 :]
    
    def get_FileSize(url):
        return os.path.getsize(url)
    
class Read_file:
    '''
    Read context of file
    '''
    def __init__(self, in_url):
        self.url = in_url
        
        # Check dieu kien file co phai la csv hay khong
        
        # Get file name from in_url
        self.fileName = file_info.get_fileName(in_url)
        
        # Get file size
        self.file_size = file_info.get_FileSize(in_url)
        
        # Get extendion of file from in url
        self.fileExtention = file_info.get_Extension(in_url)
        
        self.file_context = ''
        self.file_columns = ''
        
        if self.fileExtention == 'csv':
            self.file_context = self.__read_csv()
            self.file_columns = self.file_context.columns
        
    def __read_csv(self):
        with cd.open(self.url, "r", "Shift-JIS", "ignore") as csv_file:
            df = pd.read_table(csv_file, sep=',')
        return df
    
    def get_context(self):
        return self.file_context
    
    def get_fileName(self):
        return self.fileName
    
    def get_fileExtension(self):
        return self.fileExtention

    def get_file_Columns(self):
        return self.file_columns
    
    def get_metadata(input_pd):
        '''
        Return list of all file with extendsion is csv in folder

        WARNING : - Chua check datatype. Dinh dang Datime. Gia tri max min cua datetime
        1. Datetime => Dua ra khoang tu nam nao toi nam nao => Dua ra thong ke du lieu moi nam chiem bao nhieu %
        2. String => Dua ra duoc do dai cua string => Dua ra khoang max min ki tu
        3. Int => Dua ra gia tri Max Min

        :param fileType: DataFrame
        :return: DataFrame and add 3 columns with Category, Size and Sample Data
        '''
        # Validate input
        if isinstance(input_pd, pd.DataFrame) == False:
            print('Input values is not Dataframe')
            return

        contentFile = input_pd.copy()

        rows_nums = len(contentFile)

        # IF distinct value numbers > 10% then that values is continues data type
        CATEGORY_DEFINITION = 0.1
        TOP_UNIQUE = 10
        PRECISION = 2

        unique_values_values = []

        cols_name = []

        # Duyet toan bo columns name cua table 
        for col_name in contentFile.columns:
            # Get each columns name
            cols_name.append(col_name)

            # Get unique values
            unique_values = contentFile[str(col_name)].unique()

            # Get numbers of unique values
            unique_nums = len(unique_values)

            # Category Type
            category_type = ''

            # Get rate of each unique values
            unique_values_rate = {}

            # Get Null values
            null_nums = contentFile[col_name].isnull().sum()

            # Get Min, Max values
            min_value, max_value = None, None
            try:
                min_value = float(contentFile.dropna()[col_name].min())
                max_value = float(contentFile.dropna()[col_name].max())
            except ValueError:
                min_value, max_value = None, None
            values_range = str([min_value, max_value])

            if unique_nums > (rows_nums * CATEGORY_DEFINITION):
                unique_values = None
                unique_nums = None
                category_type = 'Continuous'
            else:
                if unique_nums == 2:
                    category_type = 'Binary'
                else:
                    category_type = 'Ordinal'

                # Dat try catch cho viec /0
                try:
                    unique_values_rate = str(pd.Series(contentFile.groupby([col_name])[col_name].count()/rows_nums *100).round(decimals = PRECISION)[:TOP_UNIQUE].to_dict())
                except ValueError:
                    unique_values_rate = None

            # Add toan bo gia tri vao List theo cau truc [<Gia tri Unique>, <So luong gia tri unique>, <Data Category>]
            add_rows = {
                        'Values_Range' : values_range,
                        'Null_nums' : null_nums,
                        'Rows_nums' : rows_nums,
                        'Unique_values' : str(unique_values), 
                        'Unique_nums' : unique_nums, 
                        'Category_types' : category_type,
                        'Unique_values_rate' : unique_values_rate
                        }

            unique_values_values.append(add_rows)

        # Return Table with all values
        return pd.DataFrame(unique_values_values, index = cols_name)


In [75]:
class Read_from_folder:
    '''
    Read data from csv file
    '''
    def __init__(self, in_folder_url):
        self.folder_url = in_folder_url
        self.csv_url = self.__read_csv_url()
        self.csv_name = self.__read_csv_name()
        self.csv_filesize = self.__filesize()
        self.csv_summarize = self.__set_summarize_data()
        self.csv_context = self.__set_content()
        self.csv_summarize_metadata = self.__set_summarize_meta_data()

    def __read_csv_url(self):
        return glob.glob(self.folder_url+'\\*.csv')
    
    def __read_csv_name(self):
        csv_name = []
        for url in self.csv_url:
            csv_name.append(file_info.get_fileName(url))
        return csv_name
    
    def __filesize(self):
        file_size = []
        for url in self.csv_url:
            file_size.append(file_info.get_FileSize(url))
        return file_size
    
    def __set_content(self):
        data = {}
        for idx, val in enumerate(self.csv_url):
            file = Read_file(val)
            data[self.csv_name[idx]] = file.get_context()
        return data
    
    def __set_summarize_data(self):
        info = { \
                    'file_name': self.csv_name,
                    'file_size': self.csv_filesize,
               }
        return pd.DataFrame(info)
    
    def __set_summarize_meta_data(self):
        data = {}
        for idx, val in enumerate(self.csv_name):
            data[idx] = Read_file.get_metadata(self.csv_context[val])
        return data
    
    def get_csv_url(self):
        '''
        Return list of all file with extendsion is csv in folder
        :param fileType: No
        :return: URL with file name
        '''        
        return self.csv_url
    
    def get_csv_filename(self):
        return self.csv_name
    
    def get_csv_filesize(self):
        return self.csv_filesize
    
    def get_summarize_info(self):
        return self.csv_summarize
    
    def get_summarize_metadata(self):
        return self.csv_summarize_metadata
    
    def get_all_dataframe(self):
        return self.csv_context

# 3. Get content of data


# 4. Export meta data into Excel file

In [6]:
class Export_metadata:
    '''
    Read file in input folder
    '''
    def __init__(self, out_url, sheet_name):
        self.URL = out_url
        self.sheet_name = sheet_name.copy()

    def write_into_sheet(self, data_input, in_sheet_name):
        data_input.to_excel(self.URL, sheet_name=in_sheet_name)

    def create_excel(self, dashboard = False):
        '''
        Create Excel file with sheet name is file in folder
        :return: Excel File
        '''
        tmp_sheet_name = self.sheet_name
        
        if dashboard:
            tmp_sheet_name.append('Dashboard')
            
        workbook = xlsxwriter.Workbook(self.URL)
        for s_name in tmp_sheet_name:
            workbook.add_worksheet(s_name)
        workbook.close()

In [81]:
# I. Tong quan

# 1.1 Define URL_PATH
URL_PATH = r'D:\02_Python\00_data\\'

# 1.2 Read data info with read_input
input_data = Read_from_folder(URL_PATH)

# 1.3 Get summarize data info
input_data.get_summarize_info()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  # This is added back by InteractiveShellApp.init_path()


,file_name,file_size
0,311-service-requests.csv,55163834
1,bikes.csv,13807
2,test.csv,28629
3,train.csv,61194
4,weather_2012.csv,512557


In [82]:
# II. Read all data into Data Panel
meta_data = input_data.get_all_dataframe()

### III. Read each data info into Pandas Dataframe

In [83]:
meta_data_summarize = input_data.get_summarize_metadata()

In [84]:
meta_data_summarize[0]

,Category_types,Null_nums,Rows_nums,Unique_nums,Unique_values,Unique_values_rate,Values_Range
Unique Key,Continuous,0,111069,NaN,None,{},"[nan, nan]"
Created Date,Continuous,0,111069,NaN,None,{},"[nan, nan]"
Closed Date,Continuous,50799,111069,NaN,None,{},"[nan, nan]"
Agency,Ordinal,0,111069,26.0,['NYPD' 'DOHMH' 'DOT' 'TLC' 'HRA' 'DSNY' 'HPD'...,"{'3-1-1': 0.08, 'CHALL': 0.07, 'COIB': 0.0, 'D...","[nan, nan]"
Agency Name,Ordinal,0,111069,135.0,['New York City Police Department'\n 'Departme...,"{'3-1-1 Call Center': 0.03, '311 Executive Off...","[nan, nan]"
Complaint Type,Ordinal,0,111069,165.0,['Noise - Street/Sidewalk' 'Illegal Parking' '...,"{'APPLIANCE': 0.4, 'Adopt-A-Basket': 0.0, 'Age...","[nan, nan]"
Descriptor,Ordinal,456,111069,789.0,['Loud Talking' 'Commercial Overnight Parking'...,"{'1 Missed Collection': 0.41, '1 or 2': 0.12, ...","[nan, nan]"
Location Type,Ordinal,32047,111069,91.0,['Street/Sidewalk' 'Club/Bar/Restaurant' 'Vaca...,"{'1-2 Family Dwelling': 0.51, '1-2 Family Mixe...","[nan, nan]"
Incident Zip,Ordinal,12262,111069,442.0,[11432.0 11378.0 10032.0 10023.0 10027.0 11372...,"{0.0: 0.0, 83.0: 0.01, 6901.0: 0.0, 7087.0: 0....","[nan, nan]"
Incident Address,Continuous,26628,111069,NaN,None,{},"[nan, nan]"


In [87]:
meta_data_summarize[1]

,Category_types,Null_nums,Rows_nums,Unique_nums,Unique_values,Unique_values_rate,Values_Range
Date,Continuous,0,310,NaN,None,{},"[nan, nan]"
Berri 1,Continuous,0,310,NaN,None,{},"[nan, nan]"
Br饕euf (donn馥s non disponibles),Ordinal,310,310,1.0,[nan],{},"[nan, nan]"
Cte-Sainte-Catherine,Continuous,0,310,NaN,None,{},"[nan, nan]"
Maisonneuve 1,Continuous,0,310,NaN,None,{},"[nan, nan]"
Maisonneuve 2,Continuous,0,310,NaN,None,{},"[nan, nan]"
du Parc,Continuous,0,310,NaN,None,{},"[nan, nan]"
Pierre-Dupuy,Continuous,0,310,NaN,None,{},"[nan, nan]"
Rachel1,Continuous,0,310,NaN,None,{},"[nan, nan]"
St-Urbain (donn馥s non disponibles),Ordinal,310,310,1.0,[nan],{},"[nan, nan]"


In [88]:
meta_data_summarize[2]

,Category_types,Null_nums,Rows_nums,Unique_nums,Unique_values,Unique_values_rate,Values_Range
PassengerId,Continuous,0,418,NaN,None,{},"[904.0, 1306.0]"
Pclass,Ordinal,0,418,3.0,[3 2 1],"{1: 25.6, 2: 22.25, 3: 52.15}","[1.0, 3.0]"
Name,Continuous,0,418,NaN,None,{},"[None, None]"
Sex,Binary,0,418,2.0,['male' 'female'],"{'female': 36.36, 'male': 63.64}","[None, None]"
Age,Continuous,86,418,NaN,None,{},"[1.0, 76.0]"
SibSp,Ordinal,0,418,7.0,[0 1 2 3 4 5 8],"{0: 67.7, 1: 26.32, 2: 3.35, 3: 0.96, 4: 0.96,...","[0.0, 3.0]"
Parch,Ordinal,0,418,8.0,[0 1 3 2 4 6 5 9],"{0: 77.51, 1: 12.44, 2: 7.89, 3: 0.72, 4: 0.48...","[0.0, 4.0]"
Ticket,Continuous,0,418,NaN,None,{},"[None, None]"
Fare,Continuous,1,418,NaN,None,{},"[0.0, 512.3292]"
Cabin,Continuous,327,418,NaN,None,{},"[None, None]"
